In [8]:
from datasets import load_from_disk
import pandas as pd
import re
from collections import Counter
import json
import matplotlib.pyplot as plt
from transformers import pipeline

from datasets import (
    Dataset,
    DatasetDict,
    Features,
    Sequence,
    Value,
    load_from_disk,
    load_metric,
)

In [7]:
 # JSON 파일 경로
file_path = '../data/raw/wikipedia_documents.json'

# JSON 파일 열기 및 데이터 로드
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 데이터를 Pandas 데이터프레임으로 변환
df = pd.DataFrame.from_dict(data, orient='index')


In [9]:
# 데이터셋 로드
dataset = load_from_disk("../data/raw/train_dataset/")
# 데이터프레임으로 변환하여 구조 파악 (필요한 경우)
train_df = pd.DataFrame(dataset['train'])
validation_df = pd.DataFrame(dataset['validation'])
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3952 entries, 0 to 3951
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   title              3952 non-null   object
 1   context            3952 non-null   object
 2   question           3952 non-null   object
 3   id                 3952 non-null   object
 4   answers            3952 non-null   object
 5   document_id        3952 non-null   int64 
 6   __index_level_0__  3952 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 216.2+ KB
None


In [17]:
train_df.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548


In [13]:
validation_df = validation_df[['context', 'id', 'question']]

In [16]:
# inference.py의 run_sparse_retrieval에서 datasete을 반환하는 구조 - inference일 때

f = Features(
        {
            "context": Value(dtype="string", id=None),
            "id": Value(dtype="string", id=None),
            "question": Value(dtype="string", id=None),
        }
    )
datasets = DatasetDict({"validation": Dataset.from_pandas(validation_df, features=f)})
datasets

DatasetDict({
    validation: Dataset({
        features: ['context', 'id', 'question'],
        num_rows: 240
    })
})

In [18]:
# DatasetDict 구조 확인 함수
def check_dataset_structure(datasets):
    # DatasetDict가 존재하는지 확인
    if isinstance(datasets, DatasetDict):
        # 각 split(validation 등) 별로 확인
        for split_name, dataset in datasets.items():
            print(f"Checking split: {split_name}")
            # features와 num_rows 확인
            features = dataset.features
            num_rows = dataset.num_rows
            print(f"Features: {list(features.keys())}")
            print(f"Number of rows: {num_rows}")
            
            # 기대하는 구조와 비교
            expected_features = ['context', 'id', 'question']
            expected_num_rows = 240
            
            # features 일치 확인
            if list(features.keys()) == expected_features:
                print("Features match expected structure.")
            else:
                print(f"Features do not match. Expected {expected_features} but got {list(features.keys())}")
            
            # num_rows 일치 확인
            if num_rows == expected_num_rows:
                print("Number of rows matches expected structure.")
            else:
                print(f"Number of rows does not match. Expected {expected_num_rows} but got {num_rows}")
    else:
        print("The input is not a DatasetDict.")
check_dataset_structure(datasets)

Checking split: validation
Features: ['context', 'id', 'question']
Number of rows: 240
Features match expected structure.
Number of rows matches expected structure.
